In [10]:
import pandas as pd

games = pd.read_csv("player_with_game_id.csv")
playerStats = pd.read_csv("player_regular_season_normalized.csv")

In [11]:
playerStats["year"] = playerStats["year"] + 1

In [12]:
playerStats = playerStats[playerStats["year"] >= 1996]

In [13]:
games['year'] = games.apply(lambda x: int(x['game_date'][0:4]), axis=1)
gameWithPlayerStats = pd.merge(games, playerStats, on=["ilkid", "year"], how='inner')
gameWithPlayerStats['game_id'].nunique()

9954

In [14]:
def getTeamAttributes(game, teams, columns):
    teamAttributes = {}
    teamIndex = ['t1','t2']
    teamAttributes['game_id'] = game.iloc[0]['game_id']
    teamAttributes['game_date'] = game.iloc[0]['game_date']
     
    for i in range(len(teams)):
        for col in columns:
            teamAttributes[teamIndex[i]+"_"+col] = game[(game["team_id_x"] == teams[i])][col].mean()
    teamAttributes["t1_team_ilkid"] = game[(game["team_id_x"] == teams[0])]["ilkid"].tolist()
    teamAttributes["t2_team_ilkid"] = game[(game["team_id_x"] == teams[1])]["ilkid"].tolist()

    if game.iloc[0]["winning_team_abbr"] == teams[0]:
        teamAttributes["winner"] = 0
    else:
        teamAttributes["winner"] = 1
    
    return teamAttributes

columns = ['gp','minutes','pts','oreb','dreb','reb','asts','stl','blk','turnover','pf','fga','fgm','fta','ftm','tpa','tpm']
aggregatedGame = pd.DataFrame(columns=['game_id', 'game_date', 't1_team_ilkid','t1_gp', 't1_minutes', 't1_pts','t1_oreb','t1_dreb','t1_reb','t1_asts','t1_stl','t1_blk','t1_turnover','t1_pf', 't1_fga', 't1_fgm', 't1_fta', 't1_ftm', 't1_tpa', 't1_tpm',
                                       't2_team_ilkid','t2_gp', 't2_minutes', 't2_pts','t2_oreb','t2_dreb','t2_reb','t2_asts','t2_stl','t2_blk','t2_turnover','t2_pf', 't2_fga', 't2_fgm', 't2_fta', 't2_ftm', 't2_tpa', 't2_tpm','winner'])
invalidGameIds = []
rows = []
for game_id in gameWithPlayerStats["game_id"].unique():
    gameDetails = gameWithPlayerStats[gameWithPlayerStats["game_id"] == game_id]
    teams = gameDetails["team_id_x"].unique().tolist()
    if len(teams) != 2:
        invalidGameIds.append(game_id)
        continue
    rows.append(getTeamAttributes(gameDetails, teams, columns))

aggregatedGame = pd.DataFrame(rows)
aggregatedGame

,game_id,game_date,t1_gp,t1_minutes,t1_pts,t1_oreb,t1_dreb,t1_reb,t1_asts,t1_stl,...,t2_pf,t2_fga,t2_fgm,t2_fta,t2_ftm,t2_tpa,t2_tpm,t1_team_ilkid,t2_team_ilkid,winner
0,29600012,1996-11-01 00:00:00,65.714286,26.708571,12.471429,1.218571,3.295714,4.515714,3.007143,0.902857,...,2.435000,9.271250,4.191250,2.415000,1.811250,2.342500,0.837500,"[JONESED01, SCOTTBY01, ROBINRU01, BLOUNCO01, O...","[MANNIDA01, KLEINJO01, CHAPMRE01, HORRYRO01, F...",0
1,29600027,1996-11-03 00:00:00,69.500000,28.388333,13.593333,1.348333,3.616667,4.966667,2.958333,0.953333,...,2.292500,7.225000,3.277500,2.331250,1.781250,1.183750,0.368750,"[JONESED01, SCOTTBY01, BLOUNCO01, ONEASH01, VA...","[MITCHSA01, GARNEKE01, CARRCH01, PARKSCH01, GU...",0
2,29600044,1996-11-06 00:00:00,69.500000,28.388333,13.593333,1.348333,3.616667,4.966667,2.958333,0.953333,...,2.917143,8.395714,4.090000,2.724286,1.991429,1.737143,0.674286,"[JONESED01, SCOTTBY01, BLOUNCO01, ONEASH01, VA...","[MASONAN01, BURRESC01, ZIDEKGE01, GOLDWAN01, D...",1
3,29600057,1996-11-08 00:00:00,69.500000,28.388333,13.593333,1.348333,3.616667,4.966667,2.958333,0.953333,...,2.876000,8.626000,4.048667,2.883333,1.999333,1.483333,0.618667,"[JONESED01, SCOTTBY01, BLOUNCO01, ONEASH01, VA...","[JONESPO01, WILLIWA01, WILLIWA01, WILLIWA01, T...",1
4,29600072,1996-11-10 00:00:00,72.000000,31.558000,15.670000,1.376000,3.986000,5.364000,3.402000,1.056000,...,2.635556,8.333333,3.866667,3.110000,2.473333,1.258889,0.441111,"[JONESED01, SCOTTBY01, ONEASH01, VANEXNI01, CA...","[LAETTCH01, LAETTCH01, LAETTCH01, BLAYLMO01, B...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9949,20500068,2005-11-10 00:00:00,57.900000,25.404000,9.576000,1.117000,2.667000,3.782000,2.502000,0.665000,...,2.602500,8.230000,3.940000,4.002500,3.172500,0.642500,0.192500,"[HARRIAL01, LUETY01, LUETY01, LUETY01, JOHNSJO...","[MAGGECO01, REBRAZE01, WILCOCH01, KAMANCH01]",1
9950,20500124,2005-11-18 00:00:00,64.500000,29.636250,11.407500,1.293750,3.170000,4.461250,2.983750,0.740000,...,2.156250,6.350000,2.686250,2.190000,1.672500,1.395000,0.532500,"[HARRIAL01, LUETY01, LUETY01, LUETY01, JOHNSJO...","[MASONDE01, ANDERCH01, BUTLERA01, NACHBBO01, N...",1
9951,20500397,2005-12-27 00:00:00,60.300000,25.570000,9.597000,1.098000,2.685000,3.780000,2.565000,0.667000,...,2.480000,8.890000,3.970000,3.085000,2.087500,0.807500,0.252500,"[HARRIAL01, LUETY01, LUETY01, LUETY01, JOHNSJO...","[WALLAGE01, BREZEPR01, ELYME01, BOGANKE01]",1
9952,20500057,2005-11-09 00:00:00,69.285714,30.732857,12.580000,1.547143,3.315714,4.861429,2.330000,0.992857,...,2.602500,8.230000,3.940000,4.002500,3.172500,0.642500,0.192500,"[ATKINCH01, THOMAET01, ARENAGI01, HAYWOBR01, J...","[MAGGECO01, REBRAZE01, WILCOCH01, KAMANCH01]",1


In [15]:
aggregatedGame.to_csv("prev_year_team_attr_by_game.csv")